In [ ]:
# Initialisation variable
conversion = 1
conversion_pixel_mm = 10
integral = 0.0
previous_error = 0.0
decalage = 0
timer_period[0] = 100


#Initialisation valeur non importante dans le programme final
l_speed = 50
r_speed = 50

motor_left_target  = 0
motor_right_target = 0

def calculate_directional_angle_2D(point1, point2, point3):
    vector1 = np.array(point1) - np.array(point2)
    vector3 = np.array(point3) - np.array(point2)

    # Calculate angles in radians
    angle1 = np.arctan2(vector1[1], vector1[0])
    angle3 = np.arctan2(vector3[1], vector3[0])

    # Calculate directional angle
    directional_angle = angle3 - angle1

    # Adjust the angle to be between -pi and pi
    directional_angle = (directional_angle + np.pi) % (2 * np.pi) - np.pi

    return directional_angle

# Changer les valeurs global,vitesse motteur,
def Reglage():
    global l_speed, r_speed, integral, previous_error,conversion,x_f,y_f,x_d,y_d,x_t,y_t

    # Déclaration des constantes PID
    kp = 1.0  # Terme proportionnel
    ki = 0.1  # Terme intégral
    kd = 0.01  # Terme dérivé
    tolerance = 0.01  # tolerance
    
    # Calculate the hypotenuse of the vector (point2 - point3)
    Hypothenus = np.linalg.norm(np.array([x_d,y_d]) - np.array([x_t,y_t]))

    # Calculate the directional angle between point1, point2, and point3
    Angle = calculate_directional_angle_2D([x_f,y_f],[x_d,y_d],[x_t,y_t])

    # Calculate decalage using sin(Angle) * Hypotenuse
    decalage = np.sin(Angle) * Hypothenus  

    if abs(decalage) > tolerance:
        # Calcul des termes PID
        P = kp * decalage
        I = ki * integral
        D = kd * (decalage - previous_error)

        # Calcul de la commande finale
        output = P + I + D
        
        l_speed = 600+(output * conversion)  # Assuming conversion is defined
        r_speed = 600-(output*conversion)
       
        # Mettre à jour les variables pour la prochaine itération
        previous_error = decalage
        integral += decalage


#Boucle principal 

@onevent
def timer0():
    global motor_left_target, motor_right_target, l_speed, r_speed  
    
    Reglage()
    # Set motor powers
    motor_left_target = round(l_speed) 
    motor_right_target = round(r_speed) 
        
